# Photon redirection

This tutorial demonstrates the basic working principles of OpenQML for continuous-variable backends. Starting with a photon in mode 1, the goal is to optimize a beam splitter to redirect the photon to mode 2 that was initially in vacuum (no photons).

### Creating the device

First we need to import OpenQML, as well as OpenQML's version of NumPy. This allows us to automatically compute gradients for functions that manipulate NumPy arrays, including quantum functions.

In [ ]:
import openqml as qm
from openqml import numpy as np
from openqml.optimize import GradientDescentOptimizer, MomentumOptimizer

Next, create a "device" to run the quantum node. We use StrawberryFields to simulate a photonic quantum processor with two quantum modes.

In [2]:
dev = qm.device('strawberryfields.fock', wires=2, cutoff_dim=10)

### Defining the quantum function

[Quantum functions](../concepts/qfuncs.rst) are a restricted subset of Python functions that must abide by the following restrictions:

1. Every line within the function contains a OpenQML quantum operation.
2. No classical processing of function parameters is allowed (i.e., no multiplication/addition/subtraction must be applied to or between parameters. This includes applying other non-quantum functions, such as NumPy functions!).
3. The function ends by returning a single OpenQML expectation value, or a list of OpenQML expectation values.

In this example, let's define a quantum function which places one photon into the first mode, before sending both modes through a beamsplitter.

In [3]:
@qm.qnode(dev)
def circuit(weights):

    qm.FockState(1, [0])
    qm.Beamsplitter(weights[0], weights[1], [0, 1])

    return qm.expectation.PhotonNumber(1)

In addition to the quantum function above following the three points specified above, note that we also include a [qnode decorator](../API/qfunc.rst) above the quantum function. This instructs OpenQML to create a [quantum node](../concepts/quantum_nodes.rst) on the computational graph using device ``dev`` (our Strawberry Fields Fock device defined above) to run this quantum function.

This function uses OpenQML to run the following quantum circuit:

<img src="figures/redirection_circuit.png">

Depending on the transmission parameter $w_0$, the photon remains in the first mode (i.e., $w_1 = n \pi$ with integer $n$), gets redirected to the second mode (i.e., $w_1 = n\pi / 2$) or is in a superposition of being in both modes. The phase parameter $w_2$ does not have any influence on the result, so we expect it to have a zero gradient and not to get updated at all.

### Defining the objective

Next, we define a cost that quantifies whether the redirection is successful. Since we want to maximize the expectation of the Fock measurement in the second mode, we minimize the negative of the circuit output.

In [4]:
def objective(weights):
    return -circuit(weights)

This objective has the following optimization landscape: 
 
 <img src="figures/redirection_landscape.png" width="450">

As stated above, the gradient does not depend on the phase parameter $w_2$. 

### Running the optimization

The initial values of the transmission and phase parameters are set to near-zero. This corresponds to an identity gate, in other words, the circuit leaves the photon in the first mode.

In [5]:
weights0 = np.array([0.01, 0.01])
print('Initial rotation angles:', weights0)

Initial rotation angles: [ 0.01  0.01]


In [6]:
o = GradientDescentOptimizer(0.5)

weights = weights0
for step in np.arange(1, 101):
    weights = o.step(objective, weights)
    if step%5==0:
        print('Objective after step {:5d}: {:.7f}'.format(step, objective(weights)) )

print()
print('Optimized rotation angles:', weights)

Objective after step     5: -0.0968138
Objective after step    10: -1.0000000
Objective after step    15: -1.0000000
Objective after step    20: -1.0000000
Objective after step    25: -1.0000000
Objective after step    30: -1.0000000
Objective after step    35: -1.0000000
Objective after step    40: -1.0000000
Objective after step    45: -1.0000000
Objective after step    50: -1.0000000
Objective after step    55: -1.0000000
Objective after step    60: -1.0000000
Objective after step    65: -1.0000000
Objective after step    70: -1.0000000
Objective after step    75: -1.0000000
Objective after step    80: -1.0000000
Objective after step    85: -1.0000000
Objective after step    90: -1.0000000
Objective after step    95: -1.0000000
Objective after step   100: -1.0000000

Optimized rotation angles: [ 1.57079633  0.01      ]


In [7]:
weights0 = np.array([-0.01, 0.01])
print('Initial rotation angles:', weights0)

o = MomentumOptimizer(0.5)

weights = weights0
for step in np.arange(1, 101):
    weights = o.step(objective, weights)
    if step%5==0:
        print('Objective after step {:5d}: {:.7f}'.format(step, objective(weights)) )

print()
print('Optimized rotation angles:', weights)

Initial rotation angles: [-0.01  0.01]
Objective after step     5: -0.4940222
Objective after step    10: -0.9698984
Objective after step    15: -0.6473168
Objective after step    20: -0.9803264
Objective after step    25: -0.9186031
Objective after step    30: -0.9320061
Objective after step    35: -0.9980251
Objective after step    40: -0.9870548
Objective after step    45: -0.9839137
Objective after step    50: -0.9974440
Objective after step    55: -0.9992436
Objective after step    60: -0.9969115
Objective after step    65: -0.9986680
Objective after step    70: -0.9999964
Objective after step    75: -0.9996112
Objective after step    80: -0.9996030
Objective after step    85: -0.9999457
Objective after step    90: -0.9999776
Objective after step    95: -0.9999214
Objective after step   100: -0.9999678

Optimized rotation angles: [-1.56512222  0.01000004]


(PLOT OPTIMIZER STEPS)